In [ ]:
import requests

In [ ]:
import getpass

user = input('Login :')
passwd = getpass.getpass("Password for " + user + " :")

In [ ]:
json = {
    'username': user,
    'password': passwd
}

In [ ]:
r = requests.post(
    'https://registre-national-entreprises.inpi.fr/api/sso/login',
    json=json)
r.status_code

In [ ]:
token = r.json()['token']

In [ ]:
token

### Recherche des documents associés à un SIREN

In [ ]:
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r

In [ ]:
siren = '300645124'

r = requests.get(
    f'https://registre-national-entreprises.inpi.fr/api/companies/{siren}/attachments',
    auth=BearerAuth(token)
)
r.status_code

In [ ]:
documents = r.json()

In [ ]:
documents.keys()

In [ ]:
documents['bilans']

In [ ]:
documents['bilans'][0]['dateCloture']

In [ ]:
from datetime import datetime, date

In [ ]:
datetime.strptime(documents['bilans'][0]['dateCloture'], '%Y-%m-%d').year

### Metadata

In [ ]:
identifier = documents['bilans'][0]['id']

In [ ]:
r = requests.get(
    f'https://registre-national-entreprises.inpi.fr/api/bilans/{identifier}',
    auth=BearerAuth(token)
)
r.status_code

In [ ]:
metadata = r.json()
metadata

### DL

In [ ]:
r = requests.get(
    f'https://registre-national-entreprises.inpi.fr/api/bilans/{identifier}/download',
    auth=BearerAuth(token)
)
r.status_code

In [ ]:
binary_file_path = 'test.pdf'
with open(binary_file_path, 'wb') as f:
    f.write(r.content)

### Test DocumentQuerier 

In [ ]:
import hvac
import os
client = hvac.Client(
    url="https://vault.lab.sspcloud.fr", token=os.environ["VAULT_TOKEN"]
)

secret = os.environ["VAULT_MOUNT"] + "/" + os.environ["VAULT_TOP_DIR"] + "/s3_creds"
mount_point, secret_path = secret.split("/", 1)
secret_dict = client.secrets.kv.read_secret_version(
    path=secret_path, mount_point=mount_point
)

os.environ["AWS_ACCESS_KEY_ID"] = secret_dict["data"]["data"][
    "ACCESS_KEY"
]
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_dict["data"]["data"][
    "SECRET_KEY"
]
try:
    del os.environ["AWS_SESSION_TOKEN"]
except KeyError:
    pass

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../ca_extract/page_selection")
sys.path.append("../ca_extract/")
sys.path.append("../")

In [ ]:
from page_selection.document_querier import DocumentQuerier

In [ ]:
querier = DocumentQuerier(username=user, password=passwd)

In [ ]:
querier.list_documents(siren)

In [ ]:
querier.query_document(siren, 2020, save_path="test.pdf", s3=False)

In [ ]:
#save_path = "projet-extraction-tableaux/test.pdf"
#querier.query_document(siren, 2020, save_path=save_path, s3=True)

In [ ]:
#sys.path.append("../ca_extract/")
from extraction.table_transformer.detector import TableTransformerDetector
from extraction.table_transformer.extractor import TableTransformerExtractor
from page_selection.page_selector import PageSelector


In [ ]:
import mlflow

def get_detector() -> TableTransformerDetector:
    """
    Load table detector model.

    Returns:
        TableTransformerDetector: detector.
    """
    detector = TableTransformerDetector(
        padding_factor=1.02,
        crop_padding_factor=1.02,
    )
    print("TableTransformerDetector loaded.")
    return detector
    
def get_extractor() -> TableTransformerExtractor:
    """
    Load table extractor model.

    Returns:
        TableTransformerExtractor: detector.
    """
    extractor = TableTransformerExtractor()
    print("TableTransformerExtractor loaded.")
    return extractor


def get_page_selector() -> PageSelector:
    """
    Load page selector.

    Returns:
    """
    model_name = "page_selection_fasttext"
    stage = "Staging"
    clf = mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")
    page_selector = PageSelector(
        clf=clf
    )
    print("Page selector loaded.")
    return page_selector

import os

os.environ['MLFLOW_TRACKING_URI'] = "https://projet-extraction-tableaux-mlflow.user.lab.sspcloud.fr/"
os.environ['MLFLOW_EXPERIMENT_NAME'] = "page_selection"
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'https://minio.lab.sspcloud.fr'

detector = get_detector()
extractor = get_extractor()
page_selector = get_page_selector()

In [ ]:
with open("test.pdf", "rb") as pdf_file:
    PDFbyte = pdf_file.read()

In [ ]:
import fitz
document = fitz.open(stream=PDFbyte, filetype="pdf")
# There can be multiple pages sometimes
page_number = page_selector.get_page_number(document)
st.write(f"Un tableau filiales et participations a été "
    f"repéré à la page {page_number + 1}.")
document.select([page_number])
